<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [IMDB2] - Reload and reuse a saved model
<!-- DESC --> Retrieving a saved model to perform a sentiment analysis (movie review)
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - The objective is to guess whether film reviews are **positive or negative** based on the analysis of the text. 
 - For this, we will use our **previously saved model**.

Original dataset can be find **[there](http://ai.stanford.edu/~amaas/data/sentiment/)**  
Note that [IMDb.com](https://imdb.com) offers several easy-to-use [datasets](https://www.imdb.com/interfaces/)  
For simplicity's sake, we'll use the dataset directly [embedded in Keras](https://www.tensorflow.org/api_docs/python/tf/keras/datasets)

## What we're going to do :

 - Preparing the data
 - Retrieve our saved model
 - Evaluate the result


## Step 1 - Init python stuff

In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.datasets.imdb as imdb

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

import os,sys,h5py,json,re

from importlib import reload

sys.path.append('..')
import fidle.pwk as pwk

datasets_dir = pwk.init('IMDB2')

**FIDLE 2020 - Practical Work Module**

Version              : 0.6.1 DEV
Notebook id          : IMDB2
Run time             : Friday 18 December 2020, 18:21:49
TensorFlow version   : 2.0.0
Keras version        : 2.2.4-tf
Datasets dir         : /home/pjluc/datasets/fidle
Running mode         : full
Update keras cache   : False
Save figs            : True
Path figs            : ./run/figs


## Step 2 : Preparing the data
### 2.1 - Our reviews :

In [2]:
reviews = [ "This film is particularly nice, a must see.",
             "Some films are great classics and cannot be ignored.",
             "This movie is just abominable and doesn't deserve to be seen!"]

### 2.2 - Retrieve dictionaries
Note : This dictionary is generated by [01-Embedding-Keras](01-Embedding-Keras.ipynb) notebook.

In [3]:
with open('./data/word_index.json', 'r') as fp:
    word_index = json.load(fp)
    index_word = {index:word for word,index in word_index.items()} 

### 2.3 - Clean, index and padd

In [4]:
max_len    = 256
vocab_size = 10000


nb_reviews = len(reviews)
x_data     = []

# ---- For all reviews
for review in reviews:
    # ---- First index must be <start>
    index_review=[1]
    # ---- For all words
    for w in review.split(' '):
        # ---- Clean it
        w_clean = re.sub(r"[^a-zA-Z0-9]", "", w)
        # ---- Not empty ?
        if len(w_clean)>0:
            # ---- Get the index
            w_index = word_index.get(w,2)
            if w_index>vocab_size : w_index=2
            # ---- Add the index if < vocab_size
            index_review.append(w_index)
    # ---- Add the indexed review
    x_data.append(index_review)    

# ---- Padding
x_data = keras.preprocessing.sequence.pad_sequences(x_data, value   = 0, padding = 'post', maxlen  = max_len)

### 2.4 - Have a look

In [5]:
def translate(x):
    return ' '.join( [index_word.get(i,'?') for i in x] )

for i in range(nb_reviews):
    imax=np.where(x_data[i]==0)[0][0]+5
    print(f'\nText review      :',    reviews[i])
    print(  f'x_train[{i:}]       :', list(x_data[i][:imax]), '(...)')
    print(  'Translation      :', translate(x_data[i][:imax]), '(...)')


Text review      : This film is particularly nice, a must see.
x_train[0]       : [1, 2, 22, 9, 572, 2, 6, 215, 2, 0, 0, 0, 0, 0] (...)
Translation      : <start> <unknown> film is particularly <unknown> a must <unknown> <pad> <pad> <pad> <pad> <pad> (...)

Text review      : Some films are great classics and cannot be ignored.
x_train[1]       : [1, 2, 108, 26, 87, 2239, 5, 566, 30, 2, 0, 0, 0, 0, 0] (...)
Translation      : <start> <unknown> films are great classics and cannot be <unknown> <pad> <pad> <pad> <pad> <pad> (...)

Text review      : This movie is just abominable and doesn't deserve to be seen!
x_train[2]       : [1, 2, 20, 9, 43, 2, 5, 152, 1833, 8, 30, 2, 0, 0, 0, 0, 0] (...)
Translation      : <start> <unknown> movie is just <unknown> and doesn't deserve to be <unknown> <pad> <pad> <pad> <pad> <pad> (...)


## Step 2 - Bring back the model

In [6]:
model = keras.models.load_model('./run/models/best_model.h5')

## Step 4 - Predict

In [7]:
y_pred   = model.predict(x_data)

#### And the winner is :

In [8]:
for i in range(nb_reviews):
    print(f'\n{reviews[i]:<70} =>',('NEGATIVE' if y_pred[i][0]<0.5 else 'POSITIVE'),f'({y_pred[i][0]:.2f})')


This film is particularly nice, a must see.                            => POSITIVE (0.56)

Some films are great classics and cannot be ignored.                   => POSITIVE (0.63)

This movie is just abominable and doesn't deserve to be seen!          => NEGATIVE (0.35)


In [9]:
pwk.end()

End time is : Friday 18 December 2020, 18:21:50
Duration is : 00:00:01 555ms
This notebook ends here


---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>